# Recipe RAG Playground

This notebook provides a playground environment for experimenting with the Recipe RAG system using Weaviate database and Claude API.

## ⚠️ Important: Cell Execution Order

**Please run the cells in order from top to bottom for the first time!** The cells have dependencies on each other:

1. **Import Libraries** (Cell 3) - Must be run first
2. **Database Setup and Verification** (Cell 6) - Checks if Weaviate is running
3. **Load Sample Recipe Data** (Cell 8) - Checks recipe count
4. **Configuration Settings** (Cell 9) - Sets up environment variables  
5. **Initialize Weaviate Client** (Cell 11) - Creates the weaviate_client
6. **Initialize Claude API Client** (Cell 13) - Creates the claude_client
7. **Run remaining cells** - Test connections and try examples

If you get "name not defined" errors, restart the kernel and run cells from the top in order.

## Notes
- Add your notes and observations here
- Document any interesting findings or experiments
- Track performance metrics and query results

## Import Necessary Libraries

Import all required libraries for connecting to Weaviate database and Claude API.

In [1]:
# Core libraries
import os
import sys
import json
from typing import List, Dict, Any, Optional
from datetime import datetime
import logging

# Data manipulation
import pandas as pd
import numpy as np

# Weaviate client
try:
    import weaviate
    from weaviate.classes.query import MetadataQuery
    print("✅ Weaviate client imported successfully")
except ImportError as e:
    print(f"❌ Failed to import Weaviate: {e}")
    print("Install with: pip install weaviate-client")

# Anthropic Claude API
try:
    import anthropic
    print("✅ Anthropic client imported successfully")
except ImportError as e:
    print(f"❌ Failed to import Anthropic: {e}")
    print("Install with: pip install anthropic")

# Environment variables
try:
    from dotenv import load_dotenv
    # Load .env file from current directory (playground folder)
    env_path = os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '.env')
    load_dotenv(env_path)
    print(f"✅ Environment variables loaded from: {env_path}")
    
    # Also check if .env file exists and show helpful message
    if os.path.exists(env_path):
        print("📝 .env file found - make sure to add your API keys")
    else:
        print("⚠️  .env file not found - creating example file")
        print("   Please add your API keys to the .env file")
except ImportError:
    print("⚠️  python-dotenv not available. Using system environment variables.")
    print("   Install with: pip install python-dotenv")

# Add database module to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'database'))

try:
    from config import WeaviateConfig, WeaviateManager
    from client import RecipeVectorDatabase, RecipeDocument
    from loader import RecipeDataLoader, MarkdownRecipeParser
    print("✅ Database modules imported successfully")
except ImportError as e:
    print(f"⚠️  Database modules not available: {e}")
    print("You can still use the raw Weaviate client")

print("\n📦 All imports completed!")

/Users/andreasliistro/Documents/Development/Projects/recipe-manager/playground/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/andreasliistro/Documents/Development/Projects/recipe-manager/playground/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/andreasliistro/Documents/Development/Projects/recipe-manager/playground/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one majo

✅ Weaviate client imported successfully
✅ Anthropic client imported successfully
✅ Environment variables loaded from: /Users/andreasliistro/Documents/Development/Projects/recipe-manager/playground/.env
📝 .env file found - make sure to add your API keys
⚠️  Database modules not available: cannot import name 'WeaviateManager' from 'config' (/Users/andreasliistro/Documents/Development/Projects/recipe-manager/database/config/__init__.py)
You can still use the raw Weaviate client

📦 All imports completed!


## Configure API Keys and Environment Variables

Set up environment variables and API keys for both Weaviate and Claude API access.

## Database Setup and Verification

Before connecting to Weaviate, let's ensure the database is properly set up and running.

In [2]:
# Database Setup and Verification
import subprocess
import time
from pathlib import Path

# load WEAVIATE_URL from .env file
WEAVIATE_URL = os.getenv("WEAVIATE_URL")
OPENAI_APIKEY = os.getenv("OPENAI_APIKEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")


def check_weaviate_running():
    """Check if Weaviate is running on the configured URL."""
    try:
        import requests
        response = requests.get(f"{WEAVIATE_URL}/v1/.well-known/ready", timeout=5)
        return response.status_code == 200
    except:
        return False

def setup_database_schema():
    """Set up the database schema if it doesn't exist."""
    try:
        # Try to use our database manager
        if 'WeaviateManager' in globals():
            manager = WeaviateManager()
            if manager.connect():
                schema_created = manager.create_schema()
                manager.disconnect()
                return schema_created
        return False
    except Exception as e:
        print(f"Error setting up schema: {e}")
        return False

print("🔍 Checking database setup...")

# 1. Check if Weaviate is running
weaviate_running = check_weaviate_running()
print(f"Weaviate Status: {'🟢 Running' if weaviate_running else '🔴 Not running'}")

if not weaviate_running:
    print("\n❌ Weaviate is not running!")
    print("To start Weaviate:")
    print("1. Open a new terminal")
    print("2. cd ../database")
    print("3. ./setup.sh start")
    print("4. Wait for it to be ready, then come back here")
else:
    print("✅ Weaviate is accessible")
    
    # 2. Check database schema
    print("\n🔍 Checking database schema...")
    schema_ready = setup_database_schema()
    
    if schema_ready:
        print("✅ Database schema is ready")
    else:
        print("⚠️  Database schema setup needed")
        print("To set up schema:")
        print("1. cd ../database")
        print("2. python -c \"from config import setup_database; setup_database()\"")

print("\n📋 Setup Checklist:")
print(f"{'✅' if weaviate_running else '❌'} Weaviate database running")
print(f"{'✅' if OPENAI_APIKEY and OPENAI_APIKEY != 'your_openai_api_key_here' else '❌'} OpenAI API key configured")
print(f"{'✅' if ANTHROPIC_API_KEY and ANTHROPIC_API_KEY != 'your_anthropic_api_key_here' else '❌'} Anthropic API key configured")

🔍 Checking database setup...
Weaviate Status: 🟢 Running
✅ Weaviate is accessible

🔍 Checking database schema...
⚠️  Database schema setup needed
To set up schema:
1. cd ../database
2. python -c "from config import setup_database; setup_database()"

📋 Setup Checklist:
✅ Weaviate database running
✅ OpenAI API key configured
✅ Anthropic API key configured


## Load Sample Recipe Data

If you don't have any recipe data loaded yet, let's load some sample recipes.

In [3]:
RECIPE_CLASS_NAME = os.getenv("RECIPE_CLASS_NAME", "Recipe")  # Default to "Recipe" if not set

# Check if we have recipe data and optionally load some
def check_recipe_data():
    """Check how many recipes are in the database."""
    try:
        if weaviate_client and weaviate_running:
            collections = weaviate_client.collections.list_all()
            if RECIPE_CLASS_NAME in collections:
                recipe_collection = weaviate_client.collections.get(RECIPE_CLASS_NAME)
                aggregate_result = recipe_collection.aggregate.over_all(total_count=True)
                return aggregate_result.total_count
    except Exception as e:
        print(f"Error checking recipe data: {e}")
    return 0

def load_sample_recipes():
    """Load a few sample recipes for testing."""
    try:
        # Check if RecipeDataLoader is available
        if 'RecipeDataLoader' not in globals():
            print("❌ RecipeDataLoader not available - please run the import cell first")
            return None
            
        # Initialize the data loader
        loader = RecipeDataLoader()
        recipes_dir = os.path.join(os.path.dirname(os.getcwd()), 'recipe-parser', 'data', 'recipes')
        
        if os.path.exists(recipes_dir):
            print(f"📂 Loading recipes from: {recipes_dir}")
            stats = loader.load_recipes_from_directory(recipes_dir, batch_size=5)  # Load just 5 for testing
            return stats
        else:
            print(f"❌ Recipe directory not found: {recipes_dir}")
            return None
            
    except Exception as e:
        print(f"Error loading sample recipes: {e}")
        import traceback
        traceback.print_exc()
        return None

# Check current recipe count
if weaviate_running:
    recipe_count = check_recipe_data()
    print(f"📊 Current recipes in database: {recipe_count}")
    
    if recipe_count == 0:
        print("\n🍳 No recipes found. Let's load some sample data...")
        print("This will help you test the RAG functionality.")
        
        # Ask user if they want to load sample data
        response = input("\nWould you like to load some sample recipes? (y/n): ").lower().strip()
        
        if response in ['y', 'yes']:
            print("🔄 Loading sample recipes...")
            stats = load_sample_recipes()
            
            if stats:
                print(f"\n✅ Loading completed!")
                print(f"Successful: {stats.get('successful', 0)}")
                print(f"Failed: {stats.get('failed', 0)}")
                if stats.get('errors'):
                    print(f"Errors: {stats['errors'][:3]}")  # Show first 3 errors
            else:
                print("\n❌ Failed to load sample recipes")
                print("Manual loading instructions:")
                print("1. cd ../database")
                print("2. python loader.py --recipes-dir ../recipe-parser/data/recipes --batch-size 10")
        else:
            print("\n📝 To load recipes manually later:")
            print("1. cd ../database")
            print("2. python loader.py --recipes-dir ../recipe-parser/data/recipes")
    else:
        print(f"✅ Found {recipe_count} recipes in database - ready to use!")
        print("\n🎉 Great! Your database has recipe data loaded.")
        print("You can now:")
        print("• Test vector search with recipe queries")
        print("• Use RAG with Claude API for recipe recommendations")
        print("• Experiment with different search patterns")
else:
    print("❌ Cannot check recipe data - Weaviate not running")

# Display availability status
print(f"\nRecipeDataLoader status: {'✅ Available' if 'RecipeDataLoader' in globals() else '❌ Not available'}")

Error checking recipe data: name 'weaviate_client' is not defined
📊 Current recipes in database: 0

🍳 No recipes found. Let's load some sample data...
This will help you test the RAG functionality.

📝 To load recipes manually later:
1. cd ../database
2. python loader.py --recipes-dir ../recipe-parser/data/recipes

RecipeDataLoader status: ❌ Not available


In [4]:
# Weaviate Configuration
WEAVIATE_URL = os.getenv('WEAVIATE_URL', 'http://localhost:8080')
WEAVIATE_API_KEY = os.getenv('WEAVIATE_API_KEY', '')
OPENAI_APIKEY = os.getenv('OPENAI_APIKEY', '')

# Claude API Configuration
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY', '')

# Recipe Database Configuration
RECIPE_CLASS_NAME = os.getenv('RECIPE_CLASS_NAME', 'Recipe')

# Additional Configuration
WEAVIATE_BATCH_SIZE = int(os.getenv('WEAVIATE_BATCH_SIZE', '100'))
WEAVIATE_TIMEOUT = int(os.getenv('WEAVIATE_TIMEOUT', '30'))
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'claude-3-haiku-20240307')

print("🔧 Configuration Settings:")
print(f"Weaviate URL: {WEAVIATE_URL}")
print(f"Weaviate API Key: {'✅ Set' if WEAVIATE_API_KEY else '❌ Not set'}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_APIKEY else '❌ Not set'}")
print(f"Anthropic API Key: {'✅ Set' if ANTHROPIC_API_KEY else '❌ Not set'}")
print(f"Recipe Class Name: {RECIPE_CLASS_NAME}")
print(f"Default Model: {DEFAULT_MODEL}")
print(f"Batch Size: {WEAVIATE_BATCH_SIZE}")

# Check if we're using placeholder values
placeholder_keys = []
if OPENAI_APIKEY == 'your_openai_api_key_here':
    placeholder_keys.append('OPENAI_APIKEY')
if ANTHROPIC_API_KEY == 'your_anthropic_api_key_here':
    placeholder_keys.append('ANTHROPIC_API_KEY')

# Warning messages
if not OPENAI_APIKEY or OPENAI_APIKEY == 'your_openai_api_key_here':
    print("\n⚠️  OpenAI API key not set properly. Vectorization features may not work.")
    print("   Add your OpenAI API key to the .env file")
    
if not ANTHROPIC_API_KEY or ANTHROPIC_API_KEY == 'your_anthropic_api_key_here':
    print("\n⚠️  Anthropic API key not set properly. Claude API features will not work.")
    print("   Add your Anthropic API key to the .env file")

if placeholder_keys:
    print(f"\n📝 Please update these placeholder values in .env file: {', '.join(placeholder_keys)}")

🔧 Configuration Settings:
Weaviate URL: http://localhost:8080
Weaviate API Key: ❌ Not set
OpenAI API Key: ✅ Set
Anthropic API Key: ✅ Set
Recipe Class Name: Recipe
Default Model: claude-3-haiku-20240307
Batch Size: 100


## Initialize Weaviate Client Connection

Create and configure the Weaviate client with proper authentication and connection settings.

In [5]:
# Initialize Weaviate client
weaviate_client = None

try:
    # Prepare headers
    headers = {}
    if OPENAI_APIKEY:
        headers["X-OpenAI-Api-Key"] = OPENAI_APIKEY
    
    # Extract host from URL
    host = WEAVIATE_URL.replace('http://', '').replace('https://', '').replace(':8080', '')
    
    # Connect to Weaviate
    if WEAVIATE_API_KEY:
        weaviate_client = weaviate.connect_to_local(
            host=host,
            port=8080,
            headers=headers,
            auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY)
        )
    else:
        weaviate_client = weaviate.connect_to_local(
            host=host,
            port=8080,
            headers=headers
        )
    
    print("✅ Weaviate client initialized successfully")
    
    # Test connection
    if weaviate_client.is_ready():
        print("✅ Weaviate connection verified")
    else:
        print("⚠️  Weaviate connection not ready")
    
except Exception as e:
    print(f"❌ Failed to initialize Weaviate client: {e}")
    print("Make sure Weaviate is running on", WEAVIATE_URL)
    weaviate_client = None

# Display current status
print(f"\nWeaviate client status: {'✅ Available' if weaviate_client else '❌ Not available'}")

✅ Weaviate client initialized successfully
✅ Weaviate connection verified

Weaviate client status: ✅ Available


## Initialize Claude API Client

Set up the Anthropic client for Claude API with proper authentication.

In [6]:
# Initialize Claude API client
claude_client = None

try:
    if ANTHROPIC_API_KEY:
        claude_client = anthropic.Anthropic(
            api_key=ANTHROPIC_API_KEY
        )
        print("✅ Claude API client initialized successfully")
    else:
        print("❌ Anthropic API key not provided. Claude client not initialized.")
        
except Exception as e:
    print(f"❌ Failed to initialize Claude API client: {e}")
    claude_client = None

✅ Claude API client initialized successfully


## Test Weaviate Connection

Verify the Weaviate connection by checking cluster status and available schemas.

In [7]:
if weaviate_client:
    try:
        # Test connection
        is_ready = weaviate_client.is_ready()
        is_live = weaviate_client.is_live()
        
        print("🔍 Weaviate Connection Status:")
        print(f"Ready: {'✅' if is_ready else '❌'} {is_ready}")
        print(f"Live: {'✅' if is_live else '❌'} {is_live}")
        
        # Get meta information
        meta = weaviate_client.get_meta()
        print(f"\n📊 Cluster Information:")
        print(f"Version: {meta.get('version', 'Unknown')}")
        print(f"Hostname: {meta.get('hostname', 'Unknown')}")
        
        # Get collections information (v4 API)
        collections = weaviate_client.collections.list_all()
        collection_names = list(collections.keys())
        
        print(f"\n🗂️  Available Collections ({len(collection_names)}):")
        for collection_name in collection_names:
            try:
                collection = weaviate_client.collections.get(collection_name)
                # Get collection config to show properties
                config = collection.config.get()
                properties_count = len(config.properties) if config.properties else 0
                description = config.description if hasattr(config, 'description') and config.description else 'No description'
                print(f"  • {collection_name}: {description} ({properties_count} properties)")
            except Exception as e:
                print(f"  • {collection_name}: Could not get details ({e})")
        
        # Check if Recipe class exists
        recipe_class_exists = RECIPE_CLASS_NAME in collection_names
        print(f"\n🍳 Recipe Collection Status: {'✅ Exists' if recipe_class_exists else '❌ Not found'}")
        
    except Exception as e:
        print(f"❌ Error testing Weaviate connection: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Weaviate client not available for testing")

🔍 Weaviate Connection Status:
Ready: ✅ True
Live: ✅ True

📊 Cluster Information:
Version: 1.26.1
Hostname: http://[::]:8080

🗂️  Available Collections (1):
  • Recipe: Recipe collection for RAG vector database (12 properties)

🍳 Recipe Collection Status: ✅ Exists


## Test Claude API Connection

Test the Claude API connection with a simple message exchange.

In [8]:
if claude_client:
    try:
        # Test with a simple message
        test_message = "Hello! Please respond with a brief greeting and confirm you're working correctly."
        
        print("🧪 Testing Claude API connection...")
        print(f"Test message: {test_message}")
        
        response = claude_client.messages.create(
            model=DEFAULT_MODEL,
            max_tokens=100,
            messages=[
                {"role": "user", "content": test_message}
            ]
        )
        
        print("\n✅ Claude API Response:")
        print(f"Model: {response.model}")
        print(f"Response: {response.content[0].text}")
        print(f"Usage: {response.usage.input_tokens} input tokens, {response.usage.output_tokens} output tokens")
        
    except Exception as e:
        print(f"❌ Error testing Claude API: {e}")
else:
    print("❌ Claude API client not available for testing")

🧪 Testing Claude API connection...
Test message: Hello! Please respond with a brief greeting and confirm you're working correctly.

✅ Claude API Response:
Model: claude-3-haiku-20240307
Response: Hello! Yes, I am working correctly and ready to assist you.
Usage: 22 input tokens, 17 output tokens


## Basic Weaviate Operations

Demonstrate basic Weaviate operations like querying data and examining collections.

In [9]:
if weaviate_client:
    try:
        # Check if Recipe collection exists and get some basic stats
        collections = weaviate_client.collections.list_all()
        print(f"📚 Available Collections: {list(collections.keys())}")
        
        if RECIPE_CLASS_NAME in collections:
            recipe_collection = weaviate_client.collections.get(RECIPE_CLASS_NAME)
            
            # Get collection stats
            try:
                aggregate_result = recipe_collection.aggregate.over_all(total_count=True)
                total_recipes = aggregate_result.total_count
                print(f"\n🍳 Recipe Collection Stats:")
                print(f"Total recipes: {total_recipes}")
                
                if total_recipes > 0:
                    # Get a few sample recipes
                    sample_recipes = recipe_collection.query.fetch_objects(limit=3)
                    
                    print(f"\n📋 Sample Recipes:")
                    for i, recipe in enumerate(sample_recipes.objects, 1):
                        props = recipe.properties
                        print(f"  {i}. {props.get('title', 'Untitled')}")
                        print(f"     Cuisine: {props.get('cuisine', 'Unknown')}")
                        print(f"     Source: {props.get('source', 'Unknown')[:50]}...")
                        print()
                        
                else:
                    print("\n📝 No recipes found in the collection")
                    print("   Use the data loader to import recipes from the recipe-parser folder")
                    
            except Exception as e:
                print(f"⚠️  Could not get collection stats: {e}")
        else:
            print(f"\n❌ Recipe collection '{RECIPE_CLASS_NAME}' not found")
            print("   Run the database setup script to create the schema")
            
    except Exception as e:
        print(f"❌ Error with Weaviate operations: {e}")
else:
    print("❌ Weaviate client not available")

📚 Available Collections: ['Recipe']

🍳 Recipe Collection Stats:
Total recipes: 17

📋 Sample Recipes:
  1. Wurst-Käse-Salat
     Cuisine: deutsch
     Source: https://www.swissmilk.ch/de/rezepte-kochideen/reze...

  2. Carbonara
     Cuisine: deutsch
     Source: https://www.recipetineats.com/carbonara/...

  3. Zutaten
     Cuisine: deutsch
     Source: https://www.eatbetter.de/rezepte/nudeln-mit-kicher...



## Basic Claude API Usage

Show basic Claude API usage for text generation and conversation.

In [10]:
if claude_client:
    try:
        # Example: Recipe suggestion
        prompt = """
        You are a helpful cooking assistant. Please suggest a simple, healthy recipe 
        that can be made in under 30 minutes with common ingredients. 
        Include a brief ingredient list and cooking steps.
        """
        
        print("🍳 Asking Claude for a recipe suggestion...")
        
        response = claude_client.messages.create(
            model=DEFAULT_MODEL,
            max_tokens=500,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        print("\n🤖 Claude's Recipe Suggestion:")
        print("=" * 50)
        print(response.content[0].text)
        print("=" * 50)
        print(f"\n📊 Token usage: {response.usage.input_tokens} input, {response.usage.output_tokens} output")
        
    except Exception as e:
        print(f"❌ Error with Claude API usage: {e}")
else:
    print("❌ Claude API client not available")

🍳 Asking Claude for a recipe suggestion...

🤖 Claude's Recipe Suggestion:
Here is a simple, healthy recipe for Grilled Chicken and Veggie Skewers that can be made in under 30 minutes with common ingredients:

Grilled Chicken and Veggie Skewers

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into 1-inch cubes
- 1 red bell pepper, cut into 1-inch pieces
- 1 zucchini, cut into 1-inch pieces
- 1 red onion, cut into 1-inch pieces
- 2 tbsp olive oil
- 1 tsp dried oregano
- 1 tsp garlic powder
- Salt and pepper to taste

Cooking Steps:
1. Preheat grill or grill pan to medium-high heat.
2. In a large bowl, combine the cubed chicken, bell pepper, zucchini, and red onion. Drizzle with olive oil and sprinkle with oregano, garlic powder, salt, and pepper. Toss to coat evenly.
3. Thread the chicken and vegetables onto skewers, alternating the ingredients.
4. Grill the skewers for 12-15 minutes, turning occasionally, until the chicken is cooked through and the vegetables are tender.
5. 

## Combined Usage Example

Create an example that combines both services - using Claude to analyze recipes from Weaviate.

In [11]:
if weaviate_client and claude_client:
    try:
        # First, search for recipes in Weaviate
        if RECIPE_CLASS_NAME in weaviate_client.collections.list_all():
            recipe_collection = weaviate_client.collections.get(RECIPE_CLASS_NAME)
            
            # Search for Italian recipes
            search_query = "Italian pasta recipes"
            print(f"🔍 Searching Weaviate for: '{search_query}'")
            
            search_results = recipe_collection.query.near_text(
                query=search_query,
                limit=3,
                return_metadata=MetadataQuery(certainty=True)
            )
            
            if search_results.objects:
                print(f"\n📚 Found {len(search_results.objects)} recipes:")
                
                # Prepare recipe data for Claude
                recipes_text = ""
                for i, result in enumerate(search_results.objects, 1):
                    props = result.properties
                    certainty = result.metadata.certainty if result.metadata else 0
                    
                    print(f"  {i}. {props.get('title', 'Untitled')} (certainty: {certainty:.2f})")
                    
                    recipes_text += f"""
Recipe {i}: {props.get('title', 'Untitled')}
Cuisine: {props.get('cuisine', 'Unknown')}
Ingredients: {props.get('ingredients', 'Not available')[:200]}...
Instructions: {props.get('instructions', 'Not available')[:200]}...

"""
                
                # Ask Claude to analyze the recipes
                analysis_prompt = f"""
                Please analyze the following recipes and provide:
                1. A brief summary of common ingredients
                2. Cooking difficulty assessment
                3. Suggestions for ingredient substitutions
                4. Which recipe you'd recommend for a beginner
                
                Recipes:
                {recipes_text}
                """
                
                print("\n🤖 Asking Claude to analyze the recipes...")
                
                analysis_response = claude_client.messages.create(
                    model=DEFAULT_MODEL,
                    max_tokens=800,
                    messages=[
                        {"role": "user", "content": analysis_prompt}
                    ]
                )
                
                print("\n📝 Claude's Recipe Analysis:")
                print("=" * 60)
                print(analysis_response.content[0].text)
                print("=" * 60)
                
            else:
                print("\n📝 No recipes found matching the search query")
                print("   Try searching for something else or load more recipe data")
                
                # Fallback: Ask Claude for recipe suggestions
                fallback_prompt = f"""
                I searched for '{search_query}' but didn't find any recipes. 
                Can you suggest 3 {search_query.lower()} with brief descriptions?
                """
                
                fallback_response = claude_client.messages.create(
                    model=DEFAULT_MODEL,
                    max_tokens=400,
                    messages=[
                        {"role": "user", "content": fallback_prompt}
                    ]
                )
                
                print("\n🤖 Claude's Alternative Suggestions:")
                print("=" * 50)
                print(fallback_response.content[0].text)
                print("=" * 50)
        else:
            print(f"❌ Recipe collection '{RECIPE_CLASS_NAME}' not found in Weaviate")
            
    except Exception as e:
        print(f"❌ Error in combined usage example: {e}")
        import traceback
        traceback.print_exc()
else:
    missing_services = []
    if not weaviate_client:
        missing_services.append("Weaviate")
    if not claude_client:
        missing_services.append("Claude API")
    
    print(f"❌ Cannot run combined example. Missing: {', '.join(missing_services)}")

🔍 Searching Weaviate for: 'Italian pasta recipes'

📚 Found 3 recipes:
  1. Erbsli-Pasta (certainty: 0.74)
  2. Carbonara (certainty: 0.72)
  3. Spaghetti mit Mönchsbart (certainty: 0.71)

🤖 Asking Claude to analyze the recipes...

📝 Claude's Recipe Analysis:
1. Summary of common ingredients:

The three recipes share some common ingredients, such as pasta (Teigwaren, Fusilli, Spaghetti), butter/olive oil, salt, and pepper. The recipes also include vegetables like peas (Erbsli) and barba di frate (also known as mâche or lamb's lettuce). Additionally, the Carbonara recipe likely includes eggs and cheese, which are common in this dish.

2. Cooking difficulty assessment:

- Erbsli-Pasta: This recipe appears to be relatively simple, with just a few steps involved in cooking the pasta and preparing the peas. The cooking time is also fairly short, making it a suitable option for beginner cooks.

- Carbonara: The Carbonara recipe is not provided, but this dish is generally considered to be of m

## Cleanup and Next Steps

Clean up connections and provide guidance for next steps.

In [12]:
# Cleanup connections
print("🧹 Cleaning up connections...")

if weaviate_client:
    try:
        weaviate_client.close()
        print("✅ Weaviate connection closed")
    except:
        pass

print("\n🎯 Next Steps:")
print("1. If Weaviate is not running, start it with: ./database/setup.sh start")
print("2. Set up your API keys in environment variables or .env file")
print("3. Load recipe data: python ../database/loader.py --recipes-dir ../recipe-parser/data/recipes")
print("4. Experiment with different search queries and Claude prompts")
print("5. Build your own RAG pipeline using the components above")

print("\n📖 Useful Resources:")
print("- Weaviate documentation: https://weaviate.io/developers/weaviate")
print("- Claude API documentation: https://docs.anthropic.com/")
print("- Recipe parser: ../recipe-parser/README.md")
print("- Database setup: ../database/README.md")

🧹 Cleaning up connections...
✅ Weaviate connection closed

🎯 Next Steps:
1. If Weaviate is not running, start it with: ./database/setup.sh start
2. Set up your API keys in environment variables or .env file
3. Load recipe data: python ../database/loader.py --recipes-dir ../recipe-parser/data/recipes
4. Experiment with different search queries and Claude prompts
5. Build your own RAG pipeline using the components above

📖 Useful Resources:
- Weaviate documentation: https://weaviate.io/developers/weaviate
- Claude API documentation: https://docs.anthropic.com/
- Recipe parser: ../recipe-parser/README.md
- Database setup: ../database/README.md


### 🎯 Ready for Experimentation
You can now build advanced RAG features like:
- Semantic recipe search
- Ingredient-based recommendations  
- Cooking instruction analysis
- Multi-modal recipe understanding
- Custom Claude prompts for cooking assistance